In [156]:
import pandas as pd
import numpy as np

In [157]:
stats24 = pd.read_csv('data/stats_player_post_2024.csv')

In [158]:
needed_cols = ['player_name', 'player_display_name', 'position', 'position_group', 'recent_team', 'passing_yards', 'passing_tds', 
               'passing_interceptions', 'sacks_suffered', 'sack_fumbles_lost', 'passing_2pt_conversions', 
               'rushing_yards', 'rushing_tds', 'rushing_fumbles_lost', 'rushing_2pt_conversions', 'receptions',
               'receiving_yards', 'receiving_tds', 'receiving_fumbles_lost', 'receiving_2pt_conversions',
               'special_teams_tds', 'def_tackles_solo', 'def_tackle_assists', 'def_tackles_for_loss', 'def_fumbles_forced',
               'def_sacks', 'def_interceptions', 'def_pass_defended', 'def_tds', 'def_fumbles', 'def_safeties',
               'fumble_recovery_opp', 'fumble_recovery_tds', 'punt_return_yards', 'kickoff_return_yards', 'fg_made', 'fg_att',
                'fg_made_distance',  'pat_made', 'pat_att']

In [159]:
stats24 = stats24[needed_cols]

In [160]:
position_groups = ['QB', 'RB', 'WR', 'TE', 'DL', 'LB', 'DB', 'ST']
stats24 = stats24[stats24['position_group'].isin(position_groups)]

In [161]:
special_abbreviations = pd.DataFrame(columns=['player_name', 'player_display_name', 'team', 'position_group'])
for _, row in stats24.iterrows():
    if pd.notna(row['player_name']):
        first = row['player_name'].split('.')[0]
        if len(first) > 1:
            special_abbreviations.loc[len(special_abbreviations)] = [row['player_name'], row['player_display_name'], row['recent_team'], row['position_group']]


In [162]:
special_abbreviations.head(len(special_abbreviations))

,player_name,player_display_name,team,position_group
0,Ca.Heyward,Cameron Heyward,PIT,DL
1,Da.Jones,DaQuan Jones,BUF,DL
2,Ji.Ward,Jihad Ward,MIN,LB
3,Er.Wilson,Eric Wilson,GB,LB
4,Ta.Johnson,Taron Johnson,BUF,DB
5,Ju.Watson,Justin Watson,KC,WR
6,Ty.Johnson,Ty Johnson,BUF,RB
7,Ja.Williams,Jameson Williams,DET,WR
8,Ca.Jones,Cam Jones,KC,LB
9,Do.Williams,Dorian Williams,BUF,LB


In [163]:
def get_special_abbreviations(stats_df):
    special_abbreviations = pd.DataFrame(columns=['player_name', 'player_display_name', 'team', 'position_group'])
    for _, row in stats_df.iterrows():
        if pd.notna(row['player_name']):
            first = row['player_name'].split('.')[0]
            if len(first) > 1:
                special_abbreviations.loc[len(special_abbreviations)] = [row['player_name'], row['player_display_name'], row['recent_team'], row['position_group']]

    print(special_abbreviations.head(len(special_abbreviations)))

In [164]:
def calculate_fantasy_points(row) -> float:
    match row['position_group']:
        case 'QB' | 'RB' | 'WR' | 'TE':
            return calculate_off_points(row)
        case 'DL' | 'LB' | 'DB':
            return calculate_def_points(row)
        

def calculate_off_points(row) -> float:
    total = 0
    # Passing
    total += row['passing_yards'] / 25
    total += row['passing_tds'] * 4
    total -= row['passing_interceptions'] * 2
    total -= row['sacks_suffered'] * 0.5
    total -= row['sack_fumbles_lost'] * 2
    total += row['passing_2pt_conversions'] * 2
    # Rushing
    total += row['rushing_yards'] / 10
    total += row['rushing_tds'] * 6
    total -= row['rushing_fumbles_lost'] * 2
    total += row['rushing_2pt_conversions'] * 2
    # Receiving
    total += row['receptions'] * 0.5
    total += row['receiving_yards'] / 10
    total += row['receiving_tds'] * 6
    total -= row['receiving_fumbles_lost'] * 2
    total += row['receiving_2pt_conversions'] * 2
    # Special Teams
    total += row['special_teams_tds'] * 15
    total += row['punt_return_yards'] / 20
    total += row['kickoff_return_yards'] / 20
    return total


def calculate_def_points(row):
    total = 0
    total += row['def_tackles_solo'] * 1
    total += row['def_tackle_assists'] * .5
    total += row['def_tackles_for_loss'] * 2
    total += row['def_fumbles_forced'] * 5
    total += row['def_sacks'] * 4
    total += row['def_interceptions'] * 7
    total += row['def_pass_defended'] * 3
    total += row['def_tds'] * 6
    total += row['def_safeties'] * 2
    total += row['fumble_recovery_opp'] * 3
    total += row['fumble_recovery_tds'] * 6
    total += row['special_teams_tds'] * 15
    total += row['punt_return_yards'] / 20
    total += row['kickoff_return_yards'] / 20
    return total

In [165]:
fantasy_points = []
for _, row in stats24.iterrows():
    points = calculate_fantasy_points(row)
    fantasy_points.append(points)
stats24['fantasy_points'] = fantasy_points

In [166]:
def scoring_summary(row):
    print(f"Player: {row['player_name']} ({row['position_group']})")
    print(f"Fantasy Points: {row['fantasy_points']}")
    match row['position_group']:
        case 'QB' | 'RB' | 'WR' | 'TE':
            print("Offensive Stats:")
            print(f"  Passing Yards: {row['passing_yards']}")
            print(f"  Passing TDs: {row['passing_tds']}")
            print(f"  Interceptions: {row['passing_interceptions']}")
            print(f"  Sacks Suffered: {row['sacks_suffered']}")
            print(f"  Sack Fumbles Lost: {row['sack_fumbles_lost']}")
            print(f"  Passing 2PT Conversions: {row['passing_2pt_conversions']}")
            print(f"  Rushing Yards: {row['rushing_yards']}")
            print(f"  Rushing TDs: {row['rushing_tds']}")
            print(f"  Rushing Fumbles Lost: {row['rushing_fumbles_lost']}")
            print(f"  Rushing 2PT Conversions: {row['rushing_2pt_conversions']}")
            print(f"  Receptions: {row['receptions']}")
            print(f"  Receiving Yards: {row['receiving_yards']}")
            print(f"  Receiving TDs: {row['receiving_tds']}")
            print(f"  Receiving Fumbles Lost: {row['receiving_fumbles_lost']}")
            print(f"  Receiving 2PT Conversions: {row['receiving_2pt_conversions']}")
            print(f"  Special Teams TDs: {row['special_teams_tds']}")
            print(f"  Punt Return Yards: {row['punt_return_yards']}")          
            print(f"  Kickoff Return Yards: {row['kickoff_return_yards']}")
        case 'DL' | 'LB' | 'DB':
            print("Defensive Stats:")
            print(f"  Solo Tackles: {row['def_tackles_solo']}")
            print(f"  Assisted Tackles: {row['def_tackle_assists']}")
            print(f"  Tackles for Loss: {row['def_tackles_for_loss']}")
            print(f"  Forced Fumbles: {row['def_fumbles_forced']}")
            print(f"  Sacks: {row['def_sacks']}")
            print(f"  Interceptions: {row['def_interceptions']}")
            print(f"  Passes Defended: {row['def_pass_defended']}")
            print(f"  Defensive TDs: {row['def_tds']}")
            print(f"  Safeties: {row['def_safeties']}")
            print(f"  Fumble Recoveries (Opp): {row['fumble_recovery_opp']}")
            print(f"  Fumble Recovery TDs: {row['fumble_recovery_tds']}")

    print(f" Fantasy Points: {row['fantasy_points']}")

In [167]:
stats_player_week_24 = pd.read_csv('data/stats_player_week_2024.csv')
weekly_stats_24 = stats_player_week_24[stats_player_week_24['season_type'] == 'POST']
needed_cols_weekly = needed_cols + ['week'] + ['team']
needed_cols_weekly.remove('recent_team')
weekly_stats_24 = weekly_stats_24[needed_cols_weekly]


In [168]:
player_dict = {}
for _, row in weekly_stats_24.iterrows():
    player = f"{row['player_display_name']}, {row['team']}, {row['position']}"
    week = row['week']
    if player not in player_dict:
        player_dict[player] = 0.0
    weekly_points = calculate_fantasy_points(row)
    if weekly_points is not None:
        player_dict[player] = player_dict[player] + weekly_points


In [169]:
fantasy_points = []
for _, row in weekly_stats_24.iterrows():
    weekly_points = calculate_fantasy_points(row)
    fantasy_points.append(weekly_points)

weekly_stats_24['fantasy_points'] = fantasy_points


In [170]:
test_rosters = pd.read_csv('data/rosters_test.csv')

In [171]:
games = pd.read_csv('data/games.csv')
games_24 = games[(games['season'] == 2024)]
games_24 = games_24[games_24['week'] >= 19]

In [172]:
len(games_24)

13

In [173]:
def get_eliminated_teams(games_df):
    eliminated_teams = set()
    for _, row in games_df.iterrows():
        if row['result'] > 0:
            eliminated_teams.add(row['away_team'])
        elif row['result'] < 0:
            eliminated_teams.add(row['home_team'])
    return eliminated_teams

In [174]:
print(get_eliminated_teams(games_24))

{'DET', 'GB', 'TB', 'PIT', 'LA', 'BUF', 'HOU', 'WAS', 'KC', 'DEN', 'LAC', 'BAL', 'MIN'}


In [175]:
def get_player_position(player_name, team, year_stats):
    player_row = year_stats[(year_stats['player_name'] == player_name) & (year_stats['recent_team'] == team)]
    if not player_row.empty:
        return player_row.iloc[0]['position_group']
    else:
        print(f"No position found for {player_name} on team {team}")
        return None

In [176]:
def get_player_fantasy_points(player_name, team_abbr, weekly_stats):
    games = weekly_stats[(weekly_stats['player_name'] == player_name) & (weekly_stats['team'] == team_abbr)].sort_values(by='week')
    total_points = 0
    out = []
    if len(games) == 0:
        print(f"No games found for {player_name} on team {team_abbr}")
    for _, row in games.iterrows():
        points = round(calculate_fantasy_points(row), 2)
        out.append(points)
        total_points += points
    if len(out) < 4:
        if not games.empty and (games.iloc[0]['team'] == 'SEA' or games.iloc[0]['team'] == 'DEN'):
            out = [0.0] + out
        out += [0.0] * (4 - len(out))
    out.append(round(total_points, 2))
    return out


In [177]:
def get_team_page(rosters, team_name, weekly_stats, yearly_stats):
    roster = rosters[team_name]
    roster_df = pd.DataFrame(columns=['Player', 'Team', 'Position', 'Wild Card', 'Divisional', 'Championship', 'Super Bowl', 'Total'])
    for player in roster:
        name, team_abbr = player.split(', ')
        points = get_player_fantasy_points(name, team_abbr, weekly_stats)
        roster_df.loc[len(roster_df)] = [name, team_abbr, get_player_position(name, team_abbr, yearly_stats)] + points
    totals = []
    offense_totals = []
    defense_totals = []
    offense = roster_df[roster_df['Position'].isin(['QB', 'RB', 'WR', 'TE'])]
    defense = roster_df[roster_df['Position'].isin(['DL', 'LB', 'DB'])]
    for col in roster_df.columns[3:]:
        totals.append(round(roster_df[col].sum(), 2))
        offense_totals.append(round(offense[col].sum(), 2))
        defense_totals.append(round(defense[col].sum(), 2))
    roster_df.loc[len(roster_df)] = ['Offense', '', ''] + offense_totals
    roster_df.loc[len(roster_df)] = ['Defense', '', ''] + defense_totals
    roster_df.loc[len(roster_df)] = ['Total', '', ''] + totals
    return roster_df


In [178]:
def get_players_remaining(roster, team, games_df, yearly_stats):
    teams_remaining = get_eliminated_teams(games_df)
    remaining = 0
    remaining_offense = 0
    remaining_defense = 0
    for player in roster[team]:
        name, team_abbr = player.split(', ')
        if team_abbr not in teams_remaining:
            remaining += 1
            position = get_player_position(name, team_abbr, yearly_stats)
            match position:
                case 'QB' | 'RB' | 'WR' | 'TE':
                    remaining_offense += 1
                case 'DL' | 'LB' | 'DB':
                    remaining_defense += 1
    return remaining, remaining_offense, remaining_defense

In [179]:
def get_league_page(rosters, weekly_stats, yearly_stats, games):
    league_df = pd.DataFrame(columns=['Team', 'Wild Card', 'Divisional', 'Champtionship', 'Super Bowl', 'Total', 'Players Remaining'])
    offense_df = pd.DataFrame(columns=['Team', 'Wild Card', 'Divisional', 'Champtionship', 'Super Bowl', 'Total', 'Players Remaining'])
    defense_df = pd.DataFrame(columns=['Team', 'Wild Card', 'Divisional', 'Champtionship', 'Super Bowl', 'Total', 'Players Remaining'])
    teams = rosters.columns.tolist()
    teams.remove('Round')
    for team_name in teams:
        team_page = get_team_page(rosters, team_name, weekly_stats, yearly_stats)
        rem, off_rem, def_rem = get_players_remaining(rosters, team_name, games, yearly_stats)
        league_df.loc[len(league_df)] = [team_name] + team_page.loc[team_page['Player'] == 'Total'].values.flatten().tolist()[3:] + [rem]
        offense_df.loc[len(offense_df)] = [team_name] + team_page.loc[team_page['Player'] == 'Offense'].values.flatten().tolist()[3:] + [off_rem]
        defense_df.loc[len(defense_df)] = [team_name] + team_page.loc[team_page['Player'] == 'Defense'].values.flatten().tolist()[3:] + [def_rem]
    return league_df, offense_df, defense_df

In [180]:
league_page, offense_page, defense_page = get_league_page(test_rosters, weekly_stats_24, stats24, games_24)
#league_page.to_csv('league_page.csv', index=False)
#offense_page.to_csv('offense_page.csv', index=False)
#defense_page.to_csv('defense_page.csv', index=False)
for team in test_rosters.columns.to_list():
    if team != 'Round':
        team_page = get_team_page(test_rosters, team, weekly_stats_24, stats24)
        #team_page.to_csv(f"{team}_page.csv", index=False)

No games found for P.Jones on team MIN
No position found for P.Jones on team MIN
No games found for Z.Flowers on team BAL
No position found for Z.Flowers on team BAL
No games found for D.Williams on team BUF
No position found for D.Williams on team BUF
No games found for P.Jones on team MIN
No position found for P.Jones on team MIN
No games found for Z.Flowers on team BAL
No position found for Z.Flowers on team BAL
No games found for D.Williams on team BUF
No position found for D.Williams on team BUF
No games found for P.Jones on team MIN
No position found for P.Jones on team MIN
No games found for Z.Flowers on team BAL
No position found for Z.Flowers on team BAL
No games found for D.Williams on team BUF
No position found for D.Williams on team BUF


In [181]:
stats_player_week_25 = pd.read_csv('data/stats_player_week_2025.csv')
weekly_stats_25 = stats_player_week_25[stats_player_week_25['season_type'] == 'POST']
needed_cols_weekly = needed_cols + ['week'] + ['team']
needed_cols_weekly.remove('recent_team')
weekly_stats_25 = weekly_stats_25[needed_cols_weekly]

C:\Users\EwEt643\AppData\Local\Temp\ipykernel_28484\2601968909.py:1: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  stats_player_week_25 = pd.read_csv('data/stats_player_week_2025.csv')


In [182]:
stats25 = pd.read_csv('data/stats_player_reg_2025.csv')

In [183]:
rosters_25 = pd.read_csv('data/rosters_25.csv')

In [184]:

games_25 = games[(games['season'] == 2025)]
games_25 = games_25[games_25['week'] >= 19]
len(games_25)

9

In [185]:
print(get_eliminated_teams(games_25))

{'GB', 'PHI', 'JAX', 'CAR', 'LAC'}



In [186]:
league_page, offense_page, defense_page = get_league_page(rosters_25, weekly_stats_25, stats25, games_25)
league_page.to_csv('league_pages/league_page.csv', index=False)
offense_page.to_csv('league_pages/offense_page.csv', index=False)
defense_page.to_csv('league_pages/defense_page.csv', index=False)
for team in rosters_25.columns.to_list():
    if team != 'Round':
        team_page = get_team_page(rosters_25,team, weekly_stats_25, stats25)
        team_page.to_csv(f"league_pages/{team}_page.csv", index=False)

No games found for N.Emmanwori on team SEA
No games found for J.Love on team SEA
No games found for J.Phillips on team PHI
No games found for P.Queen on team PIT
No games found for U.Nwosu on team SEA
No games found for C.Gardner-Johnson on team CHI
No games found for P.Wilson on team PIT
No games found for J.Ramsey on team PIT
No games found for C.Kupp on team SEA
No games found for T.Ferguson on team LA
No games found for H.To'oTo'o on team HOU
No games found for A.Singleton on team DEN
No games found for D.Stingley on team HOU
No games found for J.Campbell on team PHI
No games found for S.Darnold on team SEA
No games found for C.Sutton on team DEN
No games found for D.Schultz on team HOU
No games found for P.Freiermuth on team PIT
No games found for E.Jones on team SEA
No games found for D.Thomas on team SEA
No games found for J.Hunt on team PHI
No games found for J.Cooper on team DEN
No games found for L.Williams on team SEA
No games found for T.Togiai on team HOU
No games found fo